# Demo Notebook for the IIIF Workshop at Coding da Vinci Ost 2018 in Leipzig, Germany

https://codingdavinci.de/events/ost/

This notebook makes use of public IIIF data from the National Gallery of Art.

### Step 1: Import some libs.

In [19]:
%matplotlib inline
import numpy as np
import urllib.request
from matplotlib import pyplot as plt
import cv2
import json
from IPython.display import Image, display
from IPython.core.display import HTML 
import ipywidgets as widgets
from random import randint
import PIL

### Step 2: Read IIIF Image API endpoints from NGAs Highlights Manifest.

In [20]:
url = "https://media.nga.gov/public/manifests/nga_highlights.json"
resp = urllib.request.urlopen(url)
data = resp.read().decode("utf-8")
data = json.loads(data)
pid = 0
img_urls = {}
for c in data['sequences'][0]['canvases']:
    img_urls[pid]=c['images'][0]['resource']['service']['@id']
    print (img_urls[pid])
    pid = pid +1
    
print ("Ready.")

https://media.nga.gov/iiif/public/objects/1/0/6/3/8/2/106382-primary-0-nativeres.ptif
https://media.nga.gov/iiif/public/objects/6/1/3/7/1/61371-primary-0-nativeres.ptif
https://media.nga.gov/iiif/public/objects/5/2/1/7/8/52178-primary-0-nativeres.ptif
https://media.nga.gov/iiif/public/objects/4/6/6/2/6/46626-primary-0-nativeres.ptif
https://media.nga.gov/iiif/public/objects/1/1/3/8/1138-primary-0-nativeres.ptif
https://media.nga.gov/iiif/public/objects/4/6/1/5/9/46159-primary-0-nativeres.ptif
https://media.nga.gov/iiif/public/objects/3/3/2/5/3/33253-primary-0-nativeres.ptif
https://media.nga.gov/iiif/public/objects/1/2/3/1/1231-primary-0-nativeres.ptif
https://media.nga.gov/iiif/public/objects/3/7/0/0/3/37003-primary-0-nativeres.ptif
https://media.nga.gov/iiif/public/objects/5/7/6/576-primary-0-nativeres.ptif
https://media.nga.gov/iiif/public/objects/7/9/79-primary-0-nativeres.ptif
https://media.nga.gov/iiif/public/objects/1/2/3/6/1236-primary-0-nativeres.ptif
https://media.nga.gov/iii

### Step 3: Let's get us some generic progress bar.

In [21]:
progbar = widgets.IntProgress(
    value=0,
    min=0,
    max=pid,
    step=1,
    description='Progress:',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    orientation='horizontal'
)

### Step 4: Define a function to read a binary image from the web.

In [22]:
def url_to_image(url):
    resp = urllib.request.urlopen(url)
    image = np.asarray(bytearray(resp.read()), dtype="uint8")
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

### Step 5: Read all images with 4% of their actual size.

In [23]:
display(progbar)

img_smalls = {}
for pid in img_urls:
    progbar.value = pid
    turl = img_urls[pid]+"/full/pct:4,/0/native.jpg"
    img_smalls[pid] = url_to_image(turl)
    display(Image(url=turl, width=400, height=400))
    if pid == 1000:
        break

IntProgress(value=0, description='Progress:', max=26)

### Step 6: Get us a function to detect faces using the opencv lib.

In [24]:
def analyze_images(image):
    faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")
    faces = faceCascade.detectMultiScale(
        image,
        scaleFactor=1.1,
        minNeighbors=3,
        minSize=(20, 20),
        flags = cv2.CASCADE_SCALE_IMAGE
    )
    return faces

### Step 7: Find all faces in the images.
Save all coodinates multiplied by 25 as we retrieved the images scaled to 4%.

In [25]:
display(progbar)

faces_xy = {}
fid = 0
for pid in img_smalls:
    progbar.value=pid
    faces = analyze_images(img_smalls[pid])
    for (x, y, w, h) in faces:
        faces_xy[fid] = {}
        faces_xy[fid]['pid'] = pid
        faces_xy[fid]['x'] = x*25
        faces_xy[fid]['y'] = y*25
        faces_xy[fid]['w'] = w*25
        faces_xy[fid]['h'] = h*25
        fid = fid +1
        
print ("Found {0} faces!".format(len(faces_xy)))

IntProgress(value=25, description='Progress:', max=26)

Found 13 faces!


### Step 8: Generate URLs to the facial regions for each recognized face.

In [26]:
for fid in faces_xy:
    pid = faces_xy[fid]['pid']
    x = faces_xy[fid]['x']
    y = faces_xy[fid]['y']
    w = faces_xy[fid]['w']
    h = faces_xy[fid]['h']
    url = img_urls[pid]+"/%d,%d,%d,%d/300,/0/native.jpg"%(x,y,w,h)
    faces_xy[fid]['quick_url'] = url

### Step 9: Display what we have done so far.

In [27]:
for fid in faces_xy:
    print(faces_xy[fid]['quick_url'])
    display(Image(url=faces_xy[fid]['quick_url']))
    

https://media.nga.gov/iiif/public/objects/1/0/6/3/8/2/106382-primary-0-nativeres.ptif/2275,2475,4000,4000/300,/0/native.jpg


https://media.nga.gov/iiif/public/objects/4/6/6/2/6/46626-primary-0-nativeres.ptif/4850,1425,2850,2850/300,/0/native.jpg


https://media.nga.gov/iiif/public/objects/1/1/3/8/1138-primary-0-nativeres.ptif/18850,13750,825,825/300,/0/native.jpg


https://media.nga.gov/iiif/public/objects/1/1/3/8/1138-primary-0-nativeres.ptif/15725,17275,900,900/300,/0/native.jpg


https://media.nga.gov/iiif/public/objects/1/1/3/8/1138-primary-0-nativeres.ptif/17225,13175,925,925/300,/0/native.jpg


...

https://media.nga.gov/iiif/public/objects/3/7/0/0/3/37003-primary-0-nativeres.ptif/3750,4425,4000,4000/300,/0/native.jpg


https://media.nga.gov/iiif/public/objects/7/9/79-primary-0-nativeres.ptif/4625,3025,3725,3725/300,/0/native.jpg


https://media.nga.gov/iiif/public/objects/4/6/1/1/4/46114-primary-0-nativeres.ptif/6675,3300,2275,2275/300,/0/native.jpg


https://media.nga.gov/iiif/public/objects/1/2/1/9/8/12198-primary-0-nativeres.ptif/3675,1475,1450,1450/300,/0/native.jpg


https://media.nga.gov/iiif/public/objects/3/9/8/398-primary-0-nativeres.ptif/2450,3025,2775,2775/300,/0/native.jpg


### Use Case 1: A IIIF Memory Game

Write all face URLs to a Javascript file in order to feed them into a Memory game. This game actually uses IIIF Image API endpoints. 

In [152]:
file = open("game/urls.js","w") 
file.write("var urls=[") 
for fid in faces_xy:
    if fid > 0:
        file.write(",")
    file.write('"'+faces_xy[fid]['quick_url']+'"')
file.write("];")  
file.close() 

HTML('<a target="_blank" href="game/game.html">Game</a>')

### Use Case 2: Random Avatar Generator.

Choose randomly the upper 50% of a face, the middle 20% of another face and the lower 30% of a third face and combine it to a random patchwork portrait. Click on the image parts to flip the image horizontally.

In [153]:
efile = open("generator/eurls.js","w") 
efile.write("var eurls=[") 

nfile = open("generator/nurls.js","w") 
nfile.write("var nurls=[") 

mfile = open("generator/murls.js","w") 
mfile.write("var murls=[") 

for fid in faces_xy:
    if fid > 0:
        efile.write(",")
        nfile.write(",")
        mfile.write(",")
    u = img_urls[faces_xy[fid]['pid']]
    x = faces_xy[fid]['x']
    y = faces_xy[fid]['y']
    w = faces_xy[fid]['w']
    h = faces_xy[fid]['h']
    efile.write('"'+u+"/%d,%d,%d,%d/300,/0/native.jpg"%(x,y,w,int(h*0.5))+'"');
    nfile.write('"'+u+"/%d,%d,%d,%d/300,/0/native.jpg"%(x,y+int(h*0.5),w,int(h*0.2))+'"');
    mfile.write('"'+u+"/%d,%d,%d,%d/300,/0/native.jpg"%(x,y+int(h*0.7),w,int(h*0.3))+'"');

efile.write("];")  
efile.close() 

nfile.write("];")  
nfile.close() 

mfile.write("];")  
mfile.close() 

HTML('<a target="_blank" href="generator/index.html">Generator</a>')